In [379]:
def sub_parts(c):
    """
    For a list c = [1,2,0], return
                  [[0,0,0],
                   [1,0,0],
                   [0,1,0],
                   [1,1,0],
                   [0,2,0]]
    """
    if len(c) == 1:
        return [[x] for x in range(c[0])]
    
    if c[0] == 0:
        return [[0] + y for y in sub_parts(c[1:])]
    
    d = sub_parts(c[1:])
    g = sub_parts([c[0]-1]+c[1:])+[[c[0]]+y for y in sub_parts(c[1:])]+[[c[0]-1]+c[1:]]
    return g
    
def common_divisors(l):
    l = [a for a in l if a != 0]
    d = [set(e.divisors()) for e in l]
    f = d[0]
    for i in d[1:]:
        f = f.intersection(i)
    return f
    
def all_lists(l,s):
    #return all lists with length l and sum s
    if l == 1:
        return [[s]]
    to_return = []
    for i in range(s+1):
        to_return = to_return + [[i]+x for x in all_lists(l-1,s-i)]
    return to_return

def rm(self,depth=5):
    R = self.root_system()
    L = R.root_lattice()
    S = L.simple_roots()
    self.multiplicities = {} # Could imagine checking if it is defined first, so as
                             # not to waste time calculating things that have
                             # already been calculated.
    D = self.symmetrized_matrix().dense_matrix()
    
    def c(beta):
        dense = beta.dense_coefficient_list()
        f = common_divisors(dense)
        to_return = 0
        for e in f:
            if beta/e in self.multiplicities:
                to_return = to_return + (float(1)/float(e))*self.multiplicities[beta/e]
        return to_return
    
    def c_to_root(coeff):
        #IT IS 0 INDEXED FOR INFINITE, 1 INDEXED FOR FINITE
        #THIS IS A BUG IN SAGE
        #return sum([S[x+1]*coeff[x] for x in range(len(coeff))])#finite
        return sum([S[int(x)]*int(coeff[x]) for x in range(len(coeff))])#finite
    
    #Actually, it is SO much easier to induct on height...
    all_combs = []
    for l in [all_lists(len(S),x) for x in range(1,depth+1)]:
        all_combs = all_combs + l
    roots = [c_to_root(comb) for comb in all_combs]
    
    
    def two_rho(beta):
        return -2*sum(beta.dense_coefficient_list())
    
    
    for r in roots:
        rd = r.dense_coefficient_list()
        if sum(rd) == 1:
            self.multiplicities[r] = 1
        else:
            a_o = 0
            for bp in [c_to_root(l) for l in sub_parts(rd) if sum(l) != 0]:
                bpp = r-bp
                bpd = bp.dense_coefficient_list()
                bppd = bpp.dense_coefficient_list()
                a_o = a_o + vector(bppd)*D*vector(bpd)*c(bp)*c(bpp)
            b = vector((r).dense_coefficient_list())*D*vector((r).dense_coefficient_list()) + two_rho(r)
            
            c_o = 0
            for n in common_divisors(rd):
                if n != 1:
                    c_o = c_o + float(1)/float(n)*self.multiplicities[c_to_root(list(vector(rd)/n))]
            

            if a_o == 0 and b == 0:
                self.multiplicities[r] = 0
                continue
            self.multiplicities[r] = a_o/b-c_o
    
    return self.multiplicities
        
CartanMatrix.rm = rm
    

In [289]:
R = CartanMatrix(["E",8]) #SL3`

In [291]:
R = CartanMatrix([[2,-3],[-3,2]])

In [381]:
answer = R.rm(depth=25)

In [383]:
to_list = [[1,1],[2,2],[3,2],[3,3],[4,3],[4,4],[5,4],[6,4],[5,5],[6,5],[7,5],[6,6],[7,6],[8,6],[9,6],
          [7,7],[8,7],[9,7],[10,7],[8,8],[9,8]]
for x in to_list:
    for y in answer:
        if y.dense_coefficient_list() == x:
            print(x,answer[y])

([1, 1], 1.0)
([2, 2], 1.0)
([3, 2], 2.0)
([3, 3], 3.0)
([4, 3], 4.0)
([4, 4], 6.0)
([5, 4], 9.0)
([6, 4], 9.0)
([5, 5], 16.0)
([6, 5], 23.0)
([7, 5], 27.0)
([6, 6], 39.0)
([7, 6], 60.0)
([8, 6], 73.0)
([9, 6], 80.0)
([7, 7], 107.0)
([8, 7], 162.0)
([9, 7], 211.0)
([10, 7], 240.0)
([8, 8], 288.0)
([9, 8], 449.0)
